In [3]:
import numpy as np
import torch
import pandas as pd

In [19]:
def black_litterman(prior_mean, prior_cov, P, Q, Omega, tau=0.1):
   prior_mean = torch.tensor(prior_mean, dtype=torch.float64)
   prior_cov = torch.tensor(prior_cov, dtype=torch.float64)
   P = torch.tensor(P, dtype=torch.float64) 
   Q = torch.tensor(Q, dtype=torch.float64)
   Omega = torch.tensor(Omega, dtype=torch.float64)
   
   tau_sigma_inv = torch.linalg.inv(tau * prior_cov)
   omega_inv = torch.linalg.inv(Omega)
   
   M = torch.linalg.inv(tau_sigma_inv + P.T @ omega_inv @ P)
   posterior_mean = M @ (tau_sigma_inv @ prior_mean + P.T @ omega_inv @ Q)
   
   # Fixed views display
   views_array = np.full(len(prior_mean), np.nan)
   for i in range(P.shape[0]):
       asset_idx = np.where(P[i] == 1)[0][0]
       views_array[asset_idx] = Q[i]
   
   comparison = pd.DataFrame({
       'Prior Mean': prior_mean.numpy(),
       'Views': views_array,
       'Posterior Mean': posterior_mean.numpy()
   })
   
   return comparison

In [27]:
Omega_shock = 1
Sigma_shock = 10

n_assets = 3
prior_mean = np.array([0.1, 0.2, 0.3])
prior_cov = np.eye(n_assets) * 0.2 * Sigma_shock

# Two absolute views on first and second assets
P = np.array([
   [1.0, 0.0, 0.0],    # View on first asset return
   [0.0, 1.0, 0.0]     # View on second asset return 
])
Q = np.array([0.15, 0.25])  # Expect 15% on first, 25% on second
Omega = np.eye(2) * 0.05 * Omega_shock    # Equal uncertainty for both views

result = black_litterman(prior_mean, prior_cov, P, Q, Omega)

print("Prior Covariance:")
print(prior_cov)
print("\nView Matrix P:")
print(P)
print("\nView Returns Q:")
print(Q)
print("\nView Uncertainty Omega:")
print(Omega)
print("\nResults:")
print(result)

Prior Covariance:
[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]]

View Matrix P:
[[1. 0. 0.]
 [0. 1. 0.]]

View Returns Q:
[0.15 0.25]

View Uncertainty Omega:
[[0.05 0.  ]
 [0.   0.05]]

Results:
   Prior Mean  Views  Posterior Mean
0         0.1   0.15            0.14
1         0.2   0.25            0.24
2         0.3    NaN            0.30
